# Adjust info

This notebook is used to consider late entries, reschedule matches, and move players from and into subteams.

In [1]:
from helper_functions import setup
import numpy as np
import matplotlib.pyplot as plt
import helper_functions as hf
from helper_functions.setup.team_creation import find_best_team_to_join
from helper_functions.setup.openai_image_download import generate_all_images, save_resized_animal_images


## Register latecomers

Add all latecoming players, and add them as a reserve in each subteam for the best-fitting team.

In [2]:
player_df = setup.sanitize_and_anonymize_data(overwrite=True, verbose=False)
late = player_df[player_df["late_entry"]]
teams = hf.get_teams()
subteams = hf.ALL_SUBTEAMS.values()

for _, player in late.iterrows():
    name = player["nickname"]
    # Don't do anything if the player has already been added
    if any([team.contains_player(name) for team in teams]):
        continue
    team_to_join = teams[find_best_team_to_join(teams, player)]
    team_to_join.add_player(player, register_as_reserve=True)
    print(f"{name} joins {team_to_join.name}")
    generate_all_images([name])
    save_resized_animal_images(150)


## Move players from reserve to main team

If players are attending a sport, even if just in reserve, and are substituted in, they should be registered there.

In [6]:
sport = "running_sprints"
player_name = "Pushy Bulldog"

def move_player_from_reserve_to_subteam(player_name: str, new_subteam: int | str, sport: str, save=True):
    """Move a player from the reserve subteam in the given sport to the subteam with the provided key."""
    player_df = hf.get_players().set_index("nickname", drop=False)
    current_subteam = player_df.loc[player_name, f"subteam_{sport}"]
    player_team: str = player_df.loc[player_name, "Team"]  # type: ignore
    team_letter = player_team.replace("Team ", "")
    assert f"{sport}_{team_letter}: {new_subteam}" in hf.ALL_SUBTEAMS
    if current_subteam != "R":
        print(f"The player you've chosen ({player_name}) is currently not in a reserve team for {sport}, skipping.")
        return
    print(f"Setting new subteam '{new_subteam}' in {sport} for {player_name}")
    player_df.loc[player_name, f"subteam_{sport}"] = new_subteam
    if save:
        fpath = hf.Team.backup_path_from_letter(team_letter)
        player_df[player_df["Team"] == player_team].to_csv(fpath, index=False)
move_player_from_reserve_to_subteam("Magnificent Barracuda", 2, "volleyball")


The player you've chosen (Magnificent Barracuda) is currently not in a reserve team for volleyball, skipping.


## Reschedule Matches

Some matches need to be rescheduled. We can do that here.

In [27]:
from datetime import datetime, time
from helper_functions.setup.match_scheduling import write_match_backup_from_df
from itertools import combinations

def reschedule_match_to_new_time(match_key: str, new_time: datetime | time, check_overlaps: bool = False):
    matches = hf.get_match_df().set_index("full_key", drop=False)
    old_time = matches.loc[match_key, "start"]
    if isinstance(new_time, time):
        new_time = datetime.combine(old_time, new_time)  # type: ignore
        print(new_time)
    matches.loc[match_key, "start"] = new_time
    matches.loc[match_key, "day"] = new_time.strftime("%A")
    matches.loc[match_key, "time"] = new_time.strftime("%H:%M")
    print(f"Rewritten the time for {match_key} to {new_time.strftime("%H:%M (%A)")}.")
    write_match_backup_from_df(matches, True)
    if check_overlaps:
        matches = hf.get_matches()
        num_conflicts = np.sum([comb[0].has_hard_collision(comb[1], verbose=True) for comb in combinations(matches, 2)])
        if num_conflicts == 0:
            print("No conflicts found.")


new_start_time = datetime(2024, 4, 29, 17, 45)

reschedule_match_to_new_time("basketballA: 1B: 1", time(17, 45))
reschedule_match_to_new_time("basketballB: 1C: 1", time(18, 30))
reschedule_match_to_new_time("basketballC: 1A: 1", time(19, 15), True)


2024-04-29 17:45:00
Rewritten the time for basketballA: 1B: 1 to 17:45 (Monday).
2024-04-29 18:30:00
Rewritten the time for basketballB: 1C: 1 to 18:30 (Monday).
2024-04-29 19:15:00
Rewritten the time for basketballC: 1A: 1 to 19:15 (Monday).
No conflicts found.
